In [25]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report


In [26]:
df = pd.read_csv("sih dataset .csv")
df.head(5)


,QUESTIONS,RESULT
0,Which is the best place to go shopping in your...,irrelevant
1,What is the price of OnePlus Nord,irrelevant
2,When does the sun rise,irrelevant
3,Which place do you like to travel to,irrelevant
4,Which is your favorite animal,irrelevant


In [27]:
df['RESULT'].value_counts()


irrelevant    267
relevant      265
Name: RESULT, dtype: int64

In [28]:
df['Relevant']=df['RESULT'].apply(lambda x: 1 if x=='relevant' else 0)
df.sample(5)

,QUESTIONS,RESULT,Relevant
334,Whether MCA/ MBA Department of Arts and Scienc...,relevant,1
473,What are the different documents to be uploade...,relevant,1
399,What is National Academic Depository How to apply,relevant,1
159,What is last date to forward monthly attendance,relevant,1
133,I am opting for PG course for second time am ...,relevant,1


In [7]:
del df['relevant']

KeyError: 'relevant'

In [29]:
df

,QUESTIONS,RESULT,Relevant
0,Which is the best place to go shopping in your...,irrelevant,0
1,What is the price of OnePlus Nord,irrelevant,0
2,When does the sun rise,irrelevant,0
3,Which place do you like to travel to,irrelevant,0
4,Which is your favorite animal,irrelevant,0
...,...,...,...
527,Will the scholarship transferred in case the a...,relevant,1
528,Will the scholarship be renewed if the awardee...,relevant,1
529,Are the students who have enrolled in some co...,relevant,1
530,Can the awardee claim any other scholarship (S...,relevant,1


In [30]:
X_train, X_test, y_train, y_test = train_test_split(df['QUESTIONS'],df['Relevant'], stratify=df['Relevant'])


In [31]:
X_train.head(4)

445                   time period of SWANATH Scholarship
153    How to check the Aadhar Active status of bank ...
103    Whether a student who is recipient of Scholars...
157    When will I receive my scholarship after my ba...
Name: QUESTIONS, dtype: object

In [32]:
#BERT 
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3")

In [33]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [34]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [40]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [41]:
model.fit(X_train, y_train, epochs=15)

Epoch 1/15
13/13 [==============================] - 113s 8s/step - loss: 0.4314 - accuracy: 0.8221 - precision: 0.8556 - recall: 0.7739
Epoch 2/15
13/13 [==============================] - 104s 8s/step - loss: 0.4202 - accuracy: 0.8120 - precision: 0.8131 - recall: 0.8090
Epoch 3/15
13/13 [==============================] - 81s 6s/step - loss: 0.4014 - accuracy: 0.8296 - precision: 0.8291 - recall: 0.8291
Epoch 4/15
13/13 [==============================] - 77s 6s/step - loss: 0.3855 - accuracy: 0.8446 - precision: 0.8960 - recall: 0.7789
Epoch 5/15
13/13 [==============================] - 77s 6s/step - loss: 0.3756 - accuracy: 0.8421 - precision: 0.8542 - recall: 0.8241
Epoch 6/15
13/13 [==============================] - 78s 6s/step - loss: 0.3653 - accuracy: 0.8571 - precision: 0.8737 - recall: 0.8342
Epoch 7/15
13/13 [==============================] - 79s 6s/step - loss: 0.3597 - accuracy: 0.8622 - precision: 0.9091 - recall: 0.8040
Epoch 8/15
13/13 [==============================] - 7

In [42]:
model.evaluate(X_test, y_test)


5/5 [==============================] - 25s 5s/step - loss: 0.2956 - accuracy: 0.8947 - precision: 0.9483 - recall: 0.8333


[0.2956489026546478,
 0.8947368264198303,
 0.9482758641242981,
 0.8333333134651184]

In [43]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

5/5 [==============================] - 24s 4s/step


In [44]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       1])

In [46]:
questions = [
    "what is your name",
    "what is date today",
    "what is admission proccess?",
    "what is MOOC",
    "where are you from",
    "where did i get form?",
    "how will i get scholarship?"
]
model.predict(questions)

1/1 [==============================] - 1s 1s/step


array([[0.14889316],
       [0.34507388],
       [0.4985399 ],
       [0.64395475],
       [0.24871436],
       [0.5548014 ],
       [0.60208017]], dtype=float32)

In [51]:
model.save('model1',save_format='h5')